In [1]:
import os
import sys
import time
import torch
import numpy as np
import json
import torch.nn as nn

from utils.timingtools import now
from exp_vgg import validate_vgg19_cifar10

In [2]:
class ScalarQuantizer:
    def quantize_tensor(self, tensor, n_bits, symmetric=True):
        """Quantize tensor to specified bit precision"""
        if symmetric:
            max_val = torch.max(torch.abs(tensor))
            scale = max_val / (2**(n_bits-1) - 1) if max_val > 0 else 1.0
            quantized = torch.clamp(torch.round(tensor / scale), 
                                  -2**(n_bits-1), 2**(n_bits-1)-1)
            return quantized * scale
        else:
            min_val = torch.min(tensor)
            max_val = torch.max(tensor)
            scale = (max_val - min_val) / (2**n_bits - 1) if (max_val - min_val) > 0 else 1.0
            quantized = torch.clamp(torch.round((tensor - min_val) / scale), 
                                  0, 2**n_bits-1)
            return quantized * scale + min_val

    def quantize_per_input_channel(self, kernels, n_bits=4):
        """明确按输入通道（cin）量化（卷积核维度为[cout, cin, k_h, k_w]）"""
        quantized_kernels = torch.zeros_like(kernels)
        cin = kernels.shape[1]  # 卷积核第1维为输入通道数cin
        
        with torch.no_grad():
            # 循环输入通道维度（cin），每个输入通道单独量化
            for c_in in range(cin):
                # 提取所有输出通道中属于当前输入通道的核参数
                # 形状为 [cout, 1, k_h, k_w]
                channel_data = kernels[:, c_in:c_in+1, ...]
                quantized_channel = self.quantize_tensor(channel_data, n_bits)
                quantized_kernels[:, c_in:c_in+1, ...] = quantized_channel
                
        # 计算量化误差
        mse = torch.mean((kernels - quantized_kernels) **2).item()
        return quantized_kernels, mse

In [3]:
def compress_torch_weights(model_path, compressed_model_path, n_bits=4):
    """应用按输入通道（cin）的量化逻辑（不依赖concat_weights）"""
    # 加载模型权重
    checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
    weights = checkpoint['state_dict'] if 'state_dict' in checkpoint else checkpoint

    # 初始化量化器
    quantizer = ScalarQuantizer()
    
    # 直接遍历原始权重字典，筛选卷积层
    quantized_weights = {}  # 存储量化后的权重（按层名组织）
    total_mse = 0.0
    conv_layer_count = 0

    for layer_name, weight_tensor in weights.items():
        # 仅处理卷积层权重（形状为 [cout, cin, k_h, k_w]）
        if 'features' in layer_name and len(weight_tensor.shape) == 4:
            print(f" {layer_name}, shape: {weight_tensor.shape}")
            # 按输入通道（cin）量化当前层的卷积核
            quantized_kernel, mse = quantizer.quantize_per_input_channel(weight_tensor, n_bits)
            quantized_weights[layer_name] = quantized_kernel
            total_mse += mse
            conv_layer_count += 1
            print(f" {layer_name} 's MSE: {mse:.6f}")
        else:
            # 非卷积层权重不量化，直接保留
            quantized_weights[layer_name] = weight_tensor

    # 计算平均MSE
    avg_mse = total_mse / conv_layer_count if conv_layer_count > 0 else 0.0
    print(f"Avg MSE: {avg_mse:.6f}")

    # 保存压缩模型
    compressed_weights = {'state_dict': quantized_weights}
    os.makedirs(os.path.dirname(compressed_model_path), exist_ok=True)
    torch.save(compressed_weights, compressed_model_path)
    
    # 验证量化模型
    top1_avg = validate_vgg19_cifar10(compressed_model_path)

    # 保存结果
    result_dict = dict(
        timestamp=now(),
        precision=top1_avg,
        quantization_bits=n_bits,
        average_quantization_mse=avg_mse,
        model_name='vgg19',
        task='classification',
        dataset='cifar10',
        compression_type_base='per_input_channel_quantization',
        compressed_model_name=compressed_model_path
    )
    
    save_dir = './results'
    os.makedirs(save_dir, exist_ok=True)
    json_file_path = os.path.join(save_dir, f'result_{n_bits}bits_per_cin.json')
    with open(json_file_path, 'w') as f:
        json.dump(result_dict, f, indent=4)
    
    return top1_avg, avg_mse

In [4]:
def simulate_activation_quantization(model, n_bits=4):
    """Add hooks to simulate activation quantization in VGG model"""
    quantizer = ScalarQuantizer()
    hooks = []
    
    def quant_hook(module, input, output):
        if isinstance(output, torch.Tensor):
            return quantizer.quantize_tensor(output, n_bits)
        return output
    
    # Add hooks to convolutional and linear layers
    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            hooks.append(module.register_forward_hook(quant_hook))
    
    return model, hooks

In [5]:
def main():
    model_path = 'vgg19.pth'  
    
    # set quantization bits directly in the notebook
    n_bits = 3  
    
    print(f"Starting main quantization process...")
    print(f"Using {n_bits}-bit quantization")
    
    compressed_model_path = f'./compressed_models/vgg19_per_input_channel_{n_bits}bit.pth'

    t0 = time.time()
    
    print("\n" + "="*50)
    print("Starting weight compression")
    print("="*50)
    
    acc, mse = compress_torch_weights(
        model_path, compressed_model_path, 
        n_bits=n_bits  
    )
    
    t1 = time.time()
    
    print("\n" + "="*50)
    print("Quantization Summary")
    print("="*50)
    print(f'Processing time: {t1 - t0:.2f} seconds')
    print(f'Quantization bits: {n_bits}')
    print(f'Top-1 Accuracy: {acc:.4f}')
    print(f'Quantization MSE: {mse:.6f}')

print("Main function defined!")

Main function defined!


In [6]:
if __name__ == '__main__':
    main()

Starting main quantization process...
Using 3-bit quantization

Starting weight compression
 features.0.weight, shape: torch.Size([64, 3, 3, 3])
 features.0.weight 's MSE: 0.008227
 features.2.weight, shape: torch.Size([64, 64, 3, 3])
 features.2.weight 's MSE: 0.000519
 features.5.weight, shape: torch.Size([128, 64, 3, 3])
 features.5.weight 's MSE: 0.000546
 features.7.weight, shape: torch.Size([128, 128, 3, 3])
 features.7.weight 's MSE: 0.000320
 features.10.weight, shape: torch.Size([256, 128, 3, 3])
 features.10.weight 's MSE: 0.000325
 features.12.weight, shape: torch.Size([256, 256, 3, 3])
 features.12.weight 's MSE: 0.000143
 features.14.weight, shape: torch.Size([256, 256, 3, 3])
 features.14.weight 's MSE: 0.000142
 features.16.weight, shape: torch.Size([256, 256, 3, 3])
 features.16.weight 's MSE: 0.000170
 features.19.weight, shape: torch.Size([512, 256, 3, 3])
 features.19.weight 's MSE: 0.000156
 features.21.weight, shape: torch.Size([512, 512, 3, 3])
 features.21.weight

e:\Anaconda\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\Anaconda\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
e:\Anaconda\envs\pytorch\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Test: [0/79]	Time 15.746 (15.746)	Loss 1.2306 (1.2306)	Prec@1 79.688 (79.688)
Test: [10/79]	Time 0.473 (1.881)	Loss 1.7879 (1.3636)	Prec@1 75.000 (78.835)
Test: [20/79]	Time 0.441 (1.213)	Loss 0.9521 (1.3843)	Prec@1 82.812 (78.199)
Test: [30/79]	Time 0.503 (0.976)	Loss 0.8865 (1.3897)	Prec@1 82.031 (78.049)
Test: [40/79]	Time 0.444 (0.852)	Loss 1.8008 (1.3962)	Prec@1 76.562 (78.125)
Test: [50/79]	Time 0.469 (0.777)	Loss 1.4403 (1.4031)	Prec@1 75.781 (77.987)
Test: [60/79]	Time 0.520 (0.729)	Loss 1.3048 (1.4073)	Prec@1 78.125 (77.651)
Test: [70/79]	Time 0.470 (0.692)	Loss 1.2399 (1.4300)	Prec@1 81.250 (77.377)
 * Prec@1 77.360

Quantization Summary
Processing time: 56.49 seconds
Quantization bits: 3
Top-1 Accuracy: 77.3600
Quantization MSE: 0.000684
